In [220]:
data = pd.read_csv("/content/drive/MyDrive/ml_datasets/train.csv")
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [221]:
data.shape

(20800, 5)

In [222]:
data['label'].value_counts()

1    10413
0    10387
Name: label, dtype: int64

In [193]:
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [194]:
data = data.dropna()

In [195]:
data.shape

(18285, 5)

In [197]:
X=data.drop('label',axis=1)
y=data['label']

In [198]:
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import one_hot

In [199]:
vocab_size = 5000

In [202]:
news = X.copy()
news.reset_index(inplace=True)

In [203]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [204]:
wnl = WordNetLemmatizer()
corpus = []
for i in range(0, len(news)):
  review = re.sub('[^a-zA-z]', ' ', news['title'][i])
  review = review.lower()
  review = review.split()
  review = [wnl.lemmatize(words) for words in review if words not in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [205]:
corpus

['house dem aide even see comey letter jason chaffetz tweeted',
 'flynn hillary clinton big woman campus breitbart',
 'truth might get fired',
 'civilian killed single u airstrike identified',
 'iranian woman jailed fictional unpublished story woman stoned death adultery',
 'jackie mason hollywood would love trump bombed north korea lack trans bathroom exclusive video breitbart',
 'beno hamon win french socialist party presidential nomination new york time',
 'back channel plan ukraine russia courtesy trump associate new york time',
 'obama organizing action partner soros linked indivisible disrupt trump agenda',
 'bbc comedy sketch real housewife isi cause outrage',
 'russian researcher discover secret nazi military base treasure hunter arctic [photos]',
 'u official see link trump russia',
 'yes paid government troll social medium blog forum website',
 'major league soccer argentine find home success new york time',
 'well fargo chief abruptly step new york time',
 'anonymous donor p

In [206]:
onehot_repr = [one_hot(words, vocab_size) for words in corpus]
onehot_repr

[[4542, 4382, 333, 4968, 3036, 262, 3151, 4100, 1581, 4615],
 [4413, 54, 3157, 4080, 1366, 2202, 558],
 [3496, 2865, 937, 1040],
 [187, 2564, 97, 1114, 199, 4447],
 [3948, 1366, 4782, 3757, 1595, 3166, 1366, 210, 123, 294],
 [4199,
  4980,
  3330,
  2363,
  203,
  3888,
  776,
  2653,
  4574,
  137,
  3089,
  4331,
  3526,
  2454,
  558],
 [32, 4706, 396, 1893, 3430, 1343, 1807, 473, 4606, 265, 3581],
 [550, 577, 2753, 1679, 1438, 3645, 3888, 1379, 4606, 265, 3581],
 [2918, 3887, 3973, 4506, 2331, 4177, 2271, 301, 3888, 373],
 [4937, 2273, 619, 850, 3299, 2192, 2187, 4793],
 [2477, 3607, 1206, 2318, 3135, 1271, 545, 3548, 3180, 1656, 1333],
 [1114, 4633, 3036, 1106, 3888, 1438],
 [994, 40, 892, 1475, 1556, 3012, 4902, 1209, 4222],
 [860, 626, 4144, 1286, 1881, 2994, 2993, 4606, 265, 3581],
 [1319, 4172, 470, 3486, 3145, 4606, 265, 3581],
 [1330, 2508, 965, 1301, 1782, 4938, 4273, 397, 16, 2079],
 [2204, 150, 54],
 [2084, 1486, 4682, 2703, 3888, 3693, 4542, 558],
 [3579, 232, 3157, 3105

In [207]:
sent_len = 25
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_len)
embedded_docs

array([[   0,    0,    0, ..., 4100, 1581, 4615],
       [   0,    0,    0, ..., 1366, 2202,  558],
       [   0,    0,    0, ..., 2865,  937, 1040],
       ...,
       [   0,    0,    0, ..., 4606,  265, 3581],
       [   0,    0,    0, ..., 4980,   99, 2501],
       [   0,    0,    0, ..., 4431, 3884, 3309]], dtype=int32)

In [208]:
len(embedded_docs)

18285

In [209]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 4542, 4382,  333, 4968, 3036,  262, 3151,
       4100, 1581, 4615], dtype=int32)

In [211]:
embedding_vector_features = 50
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_features, input_length=sent_len))
model.add(LSTM(100))
model.add(Dropout(0.33))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss="binary_crossentropy", optimizer='adam', metrics='accuracy')
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 25, 50)            250000    
_________________________________________________________________
lstm_11 (LSTM)               (None, 100)               60400     
_________________________________________________________________
dropout_12 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 310,501
Trainable params: 310,501
Non-trainable params: 0
_________________________________________________________________
None


In [212]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(data['label'])

In [213]:
X_final.shape, y_final.shape

((18285, 25), (18285,))

In [214]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.20, random_state=0)

In [215]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
229/229 [==============================] - 11s 40ms/step - loss: 0.4538 - accuracy: 0.7583 - val_loss: 0.1827 - val_accuracy: 0.9212
Epoch 2/10
229/229 [==============================] - 9s 41ms/step - loss: 0.1299 - accuracy: 0.9502 - val_loss: 0.1878 - val_accuracy: 0.9234
Epoch 3/10
229/229 [==============================] - 9s 38ms/step - loss: 0.0911 - accuracy: 0.9692 - val_loss: 0.1937 - val_accuracy: 0.9243
Epoch 4/10
229/229 [==============================] - 9s 37ms/step - loss: 0.0662 - accuracy: 0.9792 - val_loss: 0.2014 - val_accuracy: 0.9226
Epoch 5/10
229/229 [==============================] - 9s 38ms/step - loss: 0.0529 - accuracy: 0.9843 - val_loss: 0.2501 - val_accuracy: 0.9171
Epoch 6/10
229/229 [==============================] - 9s 38ms/step - loss: 0.0289 - accuracy: 0.9919 - val_loss: 0.2879 - val_accuracy: 0.9237
Epoch 7/10
229/229 [==============================] - 9s 38ms/step - loss: 0.0202 - accuracy: 0.9947 - val_loss: 0.3288 - val_accuracy: 0.920

In [216]:
 from sklearn.metrics import confusion_matrix, accuracy_score

In [217]:
y_pred = model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [218]:
confusion_matrix(y_test, y_pred)

array([[1866,  174],
       [ 114, 1503]])

In [219]:
accuracy_score(y_test, y_pred)

0.9212469237079574